In [2]:
from mne.time_frequency import psd_array_multitaper, psd_array_welch
from scipy.signal import hilbert
from scipy.signal import welch
import multiprocessing as mp
from scipy.io import loadmat
from scipy.io import savemat
from scipy import signal
from fooof import FOOOF
import edgeofpy as eop
from scipy import stats
import pandas as pd
import numpy as np
import argparse
import pickle
import mne.io
import mne
import os
import glob


In [3]:
FIL_FREQ = (0.5, 40) # bandpass frequencies
THRESH_TYPE = 'both' # Fosque22: 'both'

out_dir = 'data/output/avc_std_dist/'
in_dir = 'data/input/continuous/'

# make output directory
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# load patient info and conditions

hist_x = []
hist_y = []

hist_x_raw = []
hist_y_raw = []

sub_all = []
day_all = []
condition_all = []

paths = glob.glob(f"{in_dir}*.fif")
paths.sort()

#loop over all conditions and particiants
for i, path in enumerate(paths):
    print(f"Analyzing Avlanches of {path}");
    
    sub = os.path.basename(path)[:4]
    day = os.path.basename(path)[5:9]
    condition = os.path.basename(path)[10:-8]

    #################################
    #          LOAD  DATA          #
    #################################
    raw = mne.io.read_raw_fif(path, preload=True)
    raw.interpolate_bads(reset_bads=True)
    raw.pick_types(eeg=True, meg=False, stim=False, eog=False, ecg=False, emg=False, misc=False, exclude='bads')

    data = raw.get_data()
    fs= 256
    sig_length = min(data.shape[1]/fs , 300)
    nr_channels =  data.shape[0]

    # cut data and only use first 5 min or less
    cut = np.int(sig_length*fs)
    data = data[:,:cut]

    data_filt = mne.filter.filter_data(data, sfreq=fs, l_freq=FIL_FREQ[0], h_freq=FIL_FREQ[1],verbose=False)

    # FIND DEVIATIONS FOR RELATIVE STD (Per recording)
    mean_per_chan = np.mean(data_filt, axis=1, keepdims=True)
    std_per_chan = np.std(data_filt, axis=1, keepdims=True)

    # Z-score all data
    data_z = (data_filt-mean_per_chan)/std_per_chan

    scale = np.linspace(-10,10,100)
    part_hist_y, part_hist_x = np.histogram(data_z, bins = scale)

    hist_x.append(part_hist_x)
    hist_y.append(part_hist_y)

    sub_all.append(sub)
    day_all.append(day) 
    condition_all.append(condition)

    output_df_x = pd.DataFrame(hist_x)
    output_df_Y = pd.DataFrame(hist_y)
    output_2 = pd.DataFrame({'sub':sub_all, 'day': day_all,'condition':condition_all})
    tosave = pd.concat((output_2,output_df_x,output_df_Y),axis = 1)
    tosave.to_csv(f'{out_dir}/AVC_std_varley.csv', index=False, sep=',')

    # do the same without z scores
    scale = np.linspace(-100,100,100)
    part_hist_y, part_hist_x = np.histogram(data_filt, bins = scale)

    hist_x_raw.append(part_hist_x)
    hist_y_raw.append(part_hist_y)

    output_df_x = pd.DataFrame(hist_x_raw)
    output_df_Y = pd.DataFrame(hist_y_raw)
    output_2 = pd.DataFrame({'sub':sub_all, 'day': day_all,'condition':condition_all})
    tosave = pd.concat((output_2,output_df_x,output_df_Y),axis = 1)
    tosave.to_csv(f'{out_dir}/AVC_std_varley_raw.csv', index=False, sep=',')


Analyzing Avlanches of data/input/continuous/sub0-day1-jhana-raw.fif
Opening raw data file data/input/continuous/sub0-day1-jhana-raw.fif...
    Range : 512 ... 305964 =      2.000 ...  1195.172 secs
Ready.
Reading 0 ... 305452  =      0.000 ...  1193.172 secs...
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 30 sensor positions
Interpolating 2 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_74458/1770717913.py:47: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cut = np.int(sig_length*fs)


Analyzing Avlanches of data/input/continuous/sub0-day1-mindfulness-raw.fif
Opening raw data file data/input/continuous/sub0-day1-mindfulness-raw.fif...
    Range : 1991 ... 347738 =      7.777 ...  1358.352 secs
Ready.
Reading 0 ... 345747  =      0.000 ...  1350.574 secs...
Setting channel interpolation method to {'eeg': 'spline'}.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_74458/1770717913.py:38: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads(reset_bads=True)
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_74458/1770717913.py:47: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cut = np.int(sig_length*fs)


Analyzing Avlanches of data/input/continuous/sub0-day2-jhana-raw.fif
Opening raw data file data/input/continuous/sub0-day2-jhana-raw.fif...
    Range : 512 ... 323799 =      2.000 ...  1264.840 secs
Ready.
Reading 0 ... 323287  =      0.000 ...  1262.840 secs...
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 29 sensor positions
Interpolating 3 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


...

/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_74458/1770717913.py:47: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cut = np.int(sig_length*fs)


Analyzing Avlanches of data/input/continuous/sub9-day4-jhana-raw.fif
Opening raw data file data/input/continuous/sub9-day4-jhana-raw.fif...
    Range : 8086 ... 235268 =     31.586 ...   919.016 secs
Ready.
Reading 0 ... 227182  =      0.000 ...   887.430 secs...
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 29 sensor positions
Interpolating 3 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_74458/1770717913.py:47: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cut = np.int(sig_length*fs)


Analyzing Avlanches of data/input/continuous/sub9-day4-mindfulness-raw.fif
Opening raw data file data/input/continuous/sub9-day4-mindfulness-raw.fif...
    Range : 1788 ... 355149 =      6.984 ...  1387.301 secs
Ready.
Reading 0 ... 353361  =      0.000 ...  1380.316 secs...
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 28 sensor positions
Interpolating 4 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_74458/1770717913.py:47: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cut = np.int(sig_length*fs)


In [ ]:
    # FIND DEVIATIONS FOR RELATIVE STD (Per recording)
    mean_per_chan = np.mean(data_filt, axis=1, keepdims=True)
    std_per_chan = np.std(data_filt, axis=1, keepdims=True)

    # Z-score all data
    data_z = (data_filt-mean_per_chan)/std_per_chan

In [10]:
(data_z-data_filt).shape

(32, 76800)

In [6]:
data_z.shape

(32, 76800)

In [4]:
mean_per_chan

array([[-3.33861708e-08],
       [-3.62512576e-08],
       [-2.03152794e-08],
       [-9.96404407e-09],
       [-3.21808060e-08],
       [-1.94204045e-08],
       [-1.73439026e-08],
       [-1.51230917e-08],
       [-2.54962853e-08],
       [-2.05426136e-08],
       [-2.34697762e-08],
       [-3.74774129e-09],
       [-2.71829387e-08],
       [-2.73299955e-08],
       [-4.22643437e-08],
       [-2.03470008e-09],
       [-4.54177247e-10],
       [-2.61269778e-08],
       [-1.91635591e-08],
       [-4.22504581e-08],
       [-1.57681201e-08],
       [-2.71883354e-08],
       [-2.57232493e-08],
       [-4.38849125e-08],
       [-3.67938982e-08],
       [-2.64067189e-08],
       [-4.20527952e-08],
       [-4.17651410e-08],
       [-1.57165244e-08],
       [-1.55366022e-09],
       [-1.65981932e-10],
       [ 1.95790094e-09]])